In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

D:\4Kurs\1Sem\NN\Lab3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
api_key_pinecone = os.getenv("PINECONE_API_KEY")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=api_key
)

In [4]:
pc = Pinecone(api_key=api_key_pinecone)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "nn-lab"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=spec
    )

# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index before upsert:
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}




In [20]:
import pandas as pd

coursera_df = pd.read_csv("./data/coursera_courses.csv", encoding="utf-8")
messages_df = pd.read_csv("./data/message_group_product.csv", encoding="utf-8")
netflix_df = pd.read_csv("./data/netflix_titles.csv", encoding="utf-8")

print("ДО очищення:")
print(f"Coursera: {len(coursera_df)} рядків")
print(f"Messages: {len(messages_df)} рядків")
print(f"Netflix: {len(netflix_df)} рядків")
print(f"Всього: {len(coursera_df) + len(messages_df) + len(netflix_df)}")

coursera_df = coursera_df.drop_duplicates().dropna()
messages_df = messages_df.drop_duplicates().dropna()
netflix_df = netflix_df.drop_duplicates().dropna()

print("\nПІСЛЯ видалення дублікатів і пропусків:")
print(f"Coursera: {len(coursera_df)} рядків")
print(f"Messages: {len(messages_df)} рядків")
print(f"Netflix: {len(netflix_df)} рядків")

coursera_df = coursera_df.sample(n=min(1000, len(coursera_df)), random_state=42)
messages_df = messages_df.sample(n=min(1000, len(messages_df)), random_state=42)
netflix_df = netflix_df.sample(n=min(1000, len(netflix_df)), random_state=42)

print("\nПІСЛЯ вибірки по 1000 рядків:")
print(f"Coursera: {len(coursera_df)} рядків")
print(f"Messages: {len(messages_df)} рядків")
print(f"Netflix: {len(netflix_df)} рядків")
print(f"Всього: {len(coursera_df) + len(messages_df) + len(netflix_df)}")

ДО очищення:
Coursera: 1000 рядків
Messages: 4566 рядків
Netflix: 8807 рядків
Всього: 14373

ПІСЛЯ видалення дублікатів і пропусків:
Coursera: 958 рядків
Messages: 4553 рядків
Netflix: 5332 рядків

ПІСЛЯ вибірки по 1000 рядків:
Coursera: 958 рядків
Messages: 1000 рядків
Netflix: 1000 рядків
Всього: 2958


In [21]:
coursera_df.to_csv("./data/coursera_courses_clean.csv", index=False, encoding="utf-8")
messages_df.to_csv("./data/message_group_product_clean.csv", index=False, encoding="utf-8")
netflix_df.to_csv("./data/netflix_titles_clean.csv", index=False, encoding="utf-8")

In [22]:
from langchain_community.document_loaders import CSVLoader

docs = []

csv_files = [
    "./data/coursera_courses_clean.csv",
    "./data/message_group_product_clean.csv",
    "./data/netflix_titles_clean.csv"
]

for path in csv_files:
    loader = CSVLoader(
        file_path=path,
        encoding="utf-8",
        csv_args={
            'delimiter': ',',
            'quotechar': '"'
        })
    docs.extend(loader.load())

print(docs[0].page_content[:100])
print(docs[0].metadata)

Index: course_title
Height: course_organization
Weight: course_certificate_type
None: course_time,co
{'source': './data/coursera_courses_clean.csv', 'row': 0}


In [28]:
from langchain_community.document_loaders import CSVLoader

docs = []

csv_files = [
    "./data/coursera_courses_clean.csv",
    "./data/message_group_product_clean.csv",
    "./data/netflix_titles_clean.csv"
]

for path in csv_files:
    loader = CSVLoader(
        file_path=path,
        encoding="utf-8"
    )
    docs.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = splitter.split_documents(docs)
print(f"Завантажено {len(docs)} записів")
print("\nПерший запис:")
print(docs[0].page_content[:200])
print("\nMetadata:")
print(docs[0].metadata)

Завантажено 2958 записів

Перший запис:
course_title: Six Sigma Yellow Belt
course_organization: University System of Georgia
course_certificate_type: Specialization
course_time: 3 - 6 Months
course_rating: 4.6
course_reviews_num: 8.9k
cour

Metadata:
{'source': './data/coursera_courses_clean.csv', 'row': 0}


In [29]:
namespace = "text"

docsearch = PineconeVectorStore.from_documents(
    documents=texts,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

print("Index after upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index after upsert:
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'text': {'vector_count': 4950}},
 'total_vector_count': 4950,
 'vector_type': 'dense'}




In [30]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)
    print("\n")

{'matches': [{'id': '002bef3f-5181-4cbf-8e79-5e3a0777bb30',
              'metadata': {'row': 338.0,
                           'source': './data/coursera_courses_clean.csv',
                           'text': 'course_students_enrolled: 5,325\n'
                                   'course_skills: []\n'
                                   'course_summary: []\n'
                                   'course_description: In this course, you '
                                   'will learn about Artificial Intelligence '
                                   'and Machine Learning as it applies to HR '
                                   'Management. You will explore concepts '
                                   'related to the role of data in machine '
                                   'learning, AI application, limitations of '
                                   'using data in HR decisions, and how bias '
                                   'can be mitigated using blockchain '
                    

In [39]:
retriever=docsearch.as_retriever(search_kwargs={"k": 5})

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=api_key,
    temperature=0.0
)

prompt = ChatPromptTemplate.from_template(
    "Використай наведений контекст, щоб відповісти стисло і по суті.\n"
    "Контекст:\n{context}\n\nПитання: {question}"
)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
question = "Які фантастичні фільми можна подивитися"
answer = rag_chain.invoke(question)

print("Q:", question)
print("A:", answer)

Q: Які фантастичні фільми можна подивитися
A: Серед фантастичних фільмів можна подивитися:

1. **Freaks** (2018) - Молода дівчина, захована батьком, відкриває страшні істини, контактувавши з зовнішнім світом. Рейтинг: R, тривалість: 105 хв.
   
2. **Cirque du Freak: The Vampire's Assistant** (2009) - Хлопець потрапляє до цирку з дивними персонажами і зустрічає вампіра, який змінює його життя. Рейтинг: PG-13, тривалість: 109 хв.


In [41]:
question = "Які курси по вивченю Data Science найкращі?"
answer = rag_chain.invoke(question)

print("Q:", question)
print("A:", answer)

Q: Які курси по вивченю Data Science найкращі?
A: Найкращими курсами для вивчення Data Science є:

1. **Спеціалізація від IBM** - складається з 5 онлайн-курсів, що охоплюють основи Data Science, включаючи Python, SQL, статистичний аналіз та практичні навички з реальними наборами даних.

2. **Курс з візуалізації даних** - навчає ефективно візуалізувати дані та створювати інтерактивні панелі, що допомагають зрозуміти дані та приймати обґрунтовані рішення.

3. **Програма підготовки до кар'єри в Data Science** - розвиває навички, інструменти та портфоліо для початкового рівня за 5 місяців, без попереднього досвіду в програмуванні.

Ці курси надають практичні навички, необхідні для успішної кар'єри в Data Science.


In [42]:
question = "Виведи всі плятячка чорного кольору"
answer = rag_chain.invoke(question)

print("Q:", question)
print("A:", answer)

Q: Виведи всі плятячка чорного кольору
A: Ось всі платтяка чорного кольору з наведеного контексту:

1. **CATWALK228 - Footwear-Women**
   - Опис: embellished pu slipon womens casual flats - black
   - Розміри: 4, 5, 6, 7, 8, 9
   - Ціна: Rs. 3436 (MRP: 4295, знижка 20%)

2. **CLARKS53 - Footwear-Women**
   - Опис: leather lace up womens casual pumps - black
   - Розміри: 4, 5, 6, 7, 8
   - Ціна: Rs. 8499 (MRP: 8.9, знижка 10%)

3. **CLARKS18 - Footwear-Women**
   - Опис: leather slipon womens formal pumps - black
   - Розміри: 7, 8
   - Ціна: Rs. 3899 (MRP: 6499, знижка 40%)

4. **CLARKS27 - Footwear-Women**
   - Опис: fabric slipon womens casual shoes - black
   - Розміри: 3.5, 4, 5, 6, 7, 8
   - Ціна: Rs. 2499 (MRP: 4999, знижка 50%)
